## Prerequisites
- pip install langchain langchain_community langchain_chroma
- pip install -qU langchain-openai
- export LANGCHAIN_API_KEY="..."

In [1]:
import os

os.environ['USER_AGENT'] = 'sports-buddy-basic'

In [2]:
# Initialize an OpenAI model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

docs = []
web_loader = WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/2024_Summer_Olympics",)
)
wiki_loader = WikipediaLoader('Jamaica_at_the_2024_Summer_Olympics')
loaders = [web_loader, wiki_loader]

for loader in loaders:
    docs.extend(loader.load())

In [4]:
# TODO: Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# TODO: Store documents in Chroma vector database
database = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve documents.
retriever = database.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Which programmes were dropped from the 2024 Olympics?")

/opt/anaconda3/lib/python3.12/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)
/opt/anaconda3/lib/python3.12/site-packages/posthog/client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
/opt/anaconda3/lib/python3.12/site-packages/posthog/request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


'Four events were dropped from weightlifting for the 2024 Olympics. Additionally, in canoeing, two sprint events were replaced with two slalom events. The overall event total for canoeing remained at 16.'

In [5]:
rag_chain.invoke("Was there a podium sweep in the 2024 Olympics?")

"Yes, there was one podium sweep during the 2024 Olympics. It occurred in the men's BMX race on August 2, where all three medals were won by French athletes: Joris Daudet (gold), Sylvain André (silver), and Romain Mahieu (bronze)."

In [6]:
rag_chain.invoke("What does the retrieved context say about Jamaica in the 2024 Olympics?")

'Jamaica is competing at the 2024 Summer Olympics in Paris from July 26 to August 11, marking its eighteenth appearance as an independent state. The country has qualified athletes in athletics, diving, judo, and swimming, including a total of 56 competitors. Notably, Yona Knight-Wisdom will represent Jamaica in diving, while Josh Kirlew and Sabrina Lyn will compete in swimming events.'

In [7]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [8]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [9]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [10]:
conversational_rag_chain.invoke(
    {"input": "What does the retrieved context say about Jamaica in the 2024 Olympics?"},
    config={
        "configurable": {"session_id": "sports-buddy-session"}
    },
)["answer"]

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


'Jamaica competed at the 2024 Summer Olympics in Paris, marking its eighteenth appearance as an independent state. The country qualified athletes in athletics, diving, judo, and swimming events, including one diver, Yona Knight-Wisdom, and swimmers Josh Kirlew and Sabrina Lyn. The competition took place from 26 July to 11 August 2024.'

In [11]:
conversational_rag_chain.invoke(
    {"input": "Is it their eighteenth appearance as an indpendent state?"},
    config={"configurable": {"session_id": "sports-buddy-session"}},
)["answer"]

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


"Yes, the 2024 Summer Olympics marked Jamaica's eighteenth appearance as an independent state."